In [3]:
import pandas as pd
import numpy as np
# import ydata_profiling
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import os
from math import pi
from matplotlib.path import Path
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D
import json

import warnings
warnings.filterwarnings('ignore')

## 공통 전처리

In [4]:
%%time
dtype = {
    'userID': 'int16',
    'answerCode': 'int8',
    'KnowledgeTag': 'int16'
}   

# data set load
DATA_PATH = '/opt/ml/input/data'
train_path = os.path.join(DATA_PATH, 'train_data.csv')
test_path = os.path.join(DATA_PATH, 'test_data.csv')

train_df = pd.read_csv(train_path, dtype=dtype, parse_dates=['Timestamp'])
test_df = pd.read_csv(test_path, dtype=dtype, parse_dates=['Timestamp'])

diff_train = train_df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().shift(-1)

# total_seconds(): 모든 속성을 초단위로 모아서 변환
diff_train = diff_train['Timestamp'].apply(lambda x: x.total_seconds())

train_df['elapsed'] = diff_train



test_df = pd.read_csv('/opt/ml/input/data/test_data.csv', dtype=dtype, parse_dates=['Timestamp'])


diff_test = test_df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().shift(-1)

# total_seconds(): 모든 속성을 초단위로 모아서 변환
diff_test = diff_test['Timestamp'].apply(lambda x: x.total_seconds())

test_df['elapsed'] = diff_test
test_df = test_df[test_df['answerCode'] != -1]
test_df = test_df.sort_values(by=['userID', 'Timestamp']).reset_index(drop=True)


df = pd.concat([train_df, test_df])
# df = df.drop(df[df['answerCode']== -1].index) # answerCode = -1 는 예측해야하는 값이므로 제외'Affine

import pytz
kst = pytz.timezone('Asia/Seoul')
df['Timestamp'] = df['Timestamp'].dt.tz_localize('UTC').dt.tz_convert(kst)

CPU times: user 9.8 s, sys: 716 ms, total: 10.5 s
Wall time: 10.5 s


In [5]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,elapsed
0,0,A060001001,A060000001,1,2020-03-24 09:17:11+09:00,7224,3.0
1,0,A060001002,A060000001,1,2020-03-24 09:17:14+09:00,7225,8.0
2,0,A060001003,A060000001,1,2020-03-24 09:17:22+09:00,7225,7.0
3,0,A060001004,A060000001,1,2020-03-24 09:17:29+09:00,7225,7.0
4,0,A060001005,A060000001,1,2020-03-24 09:17:36+09:00,7225,11.0
...,...,...,...,...,...,...,...
259365,7439,A040197006,A040000197,1,2020-08-21 16:39:45+09:00,2132,4721258.0
259366,7439,A040130001,A040000130,0,2020-10-15 08:07:23+09:00,8832,18.0
259367,7439,A040130002,A040000130,1,2020-10-15 08:07:41+09:00,8832,21.0
259368,7439,A040130003,A040000130,1,2020-10-15 08:08:02+09:00,8244,89.0


## Overview

+ `userID` 사용자의 고유 번호입니다. 총 7,442명의 학생이 있습니다.   
+ `assessmentItemID` 사용자가 푼 문항의 일련 번호입니다.   
+ `testId` 사용자가 푼 문항이 포함된 시험지의 일련 번호입니다.   
+ `answerCode` 사용자가 푼 문항의 정답 여부를 담고 있는 이진 (0/1) 데이터입니다.   
+ `Timestamp` 사용자가 문항을 푼 시간 정보입니다.   
+ `KnowledgeTag` 사용자가 푼 문항의 고유 태그가 담겨져 있습니다.

## 이상치 파악

## assessmentitem Id / test Id [희원, 찬웅]

### 문제번호에 따른 정답

### testcode

## timestamp [현정, 희만, … ]

## knowledgetag

### baseline에서더 많이 노출된 태그가 정답률이 더 높은가?

## testcode + knowledgetag

## 기타